In [ ]:
## This script evaluates the predictions

In [ ]:
# import packages
import os, cv2
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve, f1_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.path import Path
import matplotlib.colors as mcolors
from scipy import ndimage

In [ ]:
# load data
time_start = datetime.now()
## load submission csv
SPECIES = 'palm' ## 'palm' or 'cec'
dat_sub = pd.read_csv('../data/MTurk/MTurk_' + SPECIES + '.csv')
## load human reference
href_array = []
for file_name in os.listdir('../output/mref/' + SPECIES + '/'):
    if file_name.endswith('.csv') == False:
        continue
    href = pd.read_csv('../output/mref/' + SPECIES + '/' + file_name, index_col=0)
    href_resize = cv2.resize(np.array(href, dtype=np.uint8), (1000, 1000))
    href_array.append(href_resize)
href_array = np.array(href_array)
print('Time for loading data:', datetime.now() - time_start)

In [ ]:
# function definition
## function for transforming polygon to mask
def poly2mask(nx, ny, poly_verts):
    # Create vertex coordinates for each grid cell...
    # (<0,0> is at the top left of the grid in this system)
    x, y = np.meshgrid(np.arange(nx), np.arange(ny))
    x, y = x.flatten(), y.flatten()
    points = np.vstack((x,y)).T
    path = Path(poly_verts)
    grid = path.contains_points(points)
    grid = grid.reshape((ny,nx))
    return grid

In [ ]:
# loop over the annotators, calculate leave-one-out TP, FP, TN, FN, Number of Assignments
url_arr = np.array(dat_sub['Input.img_url'])
status_arr = np.array(dat_sub['AssignmentStatus'])
work_list = np.unique(dat_sub['WorkerId'])
print('There are', len(work_list), 'users.')
nums_dict = {}
tp_dict = {}
fp_dict = {}
tn_dict = {}
fn_dict = {}
val_dict = {}
time_start = datetime.now()
print('Start time:', time_start)
n_done = 1
for work_id in work_list:
    if n_done % 5 == 0:
        print('Task Finished:', "{:.2%}".format(n_done / len(work_list)), end="\r")
    loc = np.where(dat_sub['WorkerId'] == work_id)[0]
    nums_dict[work_id] = 0
    tp_list = []
    tn_list = []
    fp_list = []
    fn_list = []
    num_acpt = 0
    for i in loc:
        img_url = url_arr[i]
        if status_arr[i] == 'Approved':
            num_acpt += 1
        img_id = int(img_url[-8:-4]) - 1
        nums_dict[work_id] += 1
        ## obtain the estimation of this human annotator
        palm_mat = np.zeros([1000, 1000])
        polyString = dat_sub['Answer.coordinates'][i]
        polyString = polyString.replace('[', '')
        polyString = polyString.replace(']', '')
        polyString = polyString.replace('"x":', '')
        polyString = polyString.replace(',"y":', ';')
        polyString = polyString.replace(',', '')
        polyList = polyString.split('{-1;-1}')
        polyList.pop()
        for polyGon in polyList:
            elementList = polyGon.split('}{')
            ## transform element list into integers
            for i in range(len(elementList)):
                element = elementList[i].replace('{', '')
                element = element.replace('}', '')
                (x, y) = element.split(';')
                x = int(float(x)) - 1
                y = int(float(y)) - 1
                elementList[i] = (max([0, x]), max([0, y]))
            polyMask = poly2mask(1000, 1000, elementList)
            palm_mat[polyMask == True] = 1
        palm_mat = cv2.resize(np.array(palm_mat, dtype=np.uint8), (100, 100))
        pic_num = img_id // 100
        res_num = img_id % 100
        r = res_num // 10
        c = res_num - r * 10
        r1 = r * 100
        r2 = r1 + 100
        c1 = c * 100
        c2 = c1 + 100
        tp = palm_mat[href_array[pic_num][r1:r2, c1:c2] == 1].sum()
        fp = palm_mat[href_array[pic_num][r1:r2, c1:c2] == 0].sum()
        tn = (1 - palm_mat[href_array[pic_num][r1:r2, c1:c2] == 0]).sum()
        fn = (1 - palm_mat[href_array[pic_num][r1:r2, c1:c2] == 1]).sum()
        tp_list.append(tp)
        fp_list.append(fp)
        tn_list.append(tn)
        fn_list.append(fn)
    val_dict[work_id] = num_acpt / len(loc)
    tp_dict[work_id] = tp_list
    fp_dict[work_id] = fp_list
    tn_dict[work_id] = tn_list
    fn_dict[work_id] = fn_list
    n_done += 1
time_lapse = datetime.now() - time_start
print('This part took time:', time_lapse)

In [ ]:
## calculate the TPR, FPR and nums_dict for each user
nums_list = []
IoU_list = []
acc_list = []
TPR_list = []
FPR_list = []
prec_list = []
rec_list = []
val_list = []
for work_id in nums_dict.keys():
    if nums_dict[work_id] == 0:
        continue
    else:
        val_list.append(val_dict[work_id])
        nums_list.append(nums_dict[work_id])
        ## calculate IoU
        if np.sum(tp_dict[work_id]) + np.sum(fp_dict[work_id]) + np.sum(fn_dict[work_id]) > 0:
            IoU = np.sum(tp_dict[work_id]) / (np.sum(tp_dict[work_id]) + np.sum(fp_dict[work_id]) + np.sum(fn_dict[work_id]))
        else:
            IoU = 0
        ## calculate Accuracy
        acc = (np.sum(tp_dict[work_id]) + np.sum(tn_dict[work_id])) / (np.sum(tp_dict[work_id]) + np.sum(fp_dict[work_id]) + np.sum(fn_dict[work_id]) + np.sum(tn_dict[work_id]))
        ## calculate TPR
        if (np.sum(tp_dict[work_id]) + np.sum(fn_dict[work_id])) > 0:
            TPR = np.sum(tp_dict[work_id]) / (np.sum(tp_dict[work_id]) + np.sum(fn_dict[work_id]))
        else:
            TPR = 0
        ## calculate FPR
        FPR = np.sum(fp_dict[work_id]) / (np.sum(fp_dict[work_id]) + np.sum(tn_dict[work_id]))
        ## calculate precision
        if (np.sum(fp_dict[work_id]) + np.sum(tp_dict[work_id])) > 0:
            prec = np.sum(tp_dict[work_id]) / (np.sum(fp_dict[work_id]) + np.sum(tp_dict[work_id]))
        else:
            prec = 0
        ## calculate recall
        if (np.sum(fn_dict[work_id]) + np.sum(tp_dict[work_id])) > 0:
            rec = np.sum(tp_dict[work_id]) / (np.sum(fn_dict[work_id]) + np.sum(tp_dict[work_id]))
        else:
            rec = 0
        ## update lists
        IoU_list.append(IoU)
        acc_list.append(acc)
        TPR_list.append(TPR)
        FPR_list.append(FPR)
        prec_list.append(prec)
        rec_list.append(rec)

In [ ]:
# evaluate the predictions
## load APL predictions
preds_array = []
k = np.ones((10, 10)) / 100 ## weight kernel ==> average predictions on overlapped areas
for file_name in os.listdir('../output/npy/npy_20_25/'):
    if file_name.endswith('.npy') == False:
        continue
    preds = np.load('../output/npy/npy_20_25/' + file_name)
    preds_new = np.zeros((1000, 1000, preds.shape[2]))
    for i in range(preds.shape[2]):
        preds_new[:, :, i] = cv2.resize(preds[:, :, i], (1000, 1000))
        preds_new[:, :, i] = ndimage.convolve(preds_new[:, :, i], k, mode='constant', cval=0.0)
    preds_array.append(preds_new)
preds_array = np.array(preds_array)

In [ ]:
## load Basic predictions
preds_basic = []
k = np.ones((10, 10)) / 100 ## weight kernel ==> average predictions on overlapped areas
for file_name in os.listdir('../output/npy/npy_Basic/'):
    if file_name.endswith('.npy') == False:
        continue
    preds = np.load('../output/npy/npy_Basic/' + file_name)
    preds_new = np.zeros((1000, 1000, preds.shape[2]))
    for i in range(preds.shape[2]):
        preds_new[:, :, i] = cv2.resize(preds[:, :, i], (1000, 1000))
        preds_new[:, :, i] = ndimage.convolve(preds_new[:, :, i], k, mode='constant', cval=0.0)
    preds_basic.append(preds_new)
preds_basic = np.array(preds_basic)

In [ ]:
# plot ROC curve
## vectorize
testy = np.array([href_array[i] for i in range(3)], dtype=np.float).flatten()
probs_apl = np.array([preds_array[i, :, :, 0] for i in range(3)]).flatten()
probs_basic = np.array([preds_basic[i, :, :, 0] for i in range(3)]).flatten()
## calculate the ROC curves
fpr_palm, tpr_palm, _ = roc_curve(testy, probs_apl)
roc_auc_palm = auc(fpr_palm, tpr_palm)
fpr_cec, tpr_cec, _ = roc_curve(testy, probs_basic)
roc_auc_cec = auc(fpr_cec, tpr_cec)
## plot the figure
lw = 2
font_size = 15
normalize = mcolors.Normalize(vmin=0, vmax=1)
colormap = cm.coolwarm
scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array([])
plt.figure(figsize=(6, 5))
plt.scatter(FPR_list, TPR_list, s = np.sqrt(nums_list)*3, label='Annotators', c= colormap(normalize(val_list)))
plt.plot(fpr_palm, tpr_palm, color='darkgreen',
         lw=lw, label='APL (AUC = %0.2f)' % roc_auc_palm)
plt.plot(fpr_cec, tpr_cec, color='darkorange',
         lw=lw, label='Basic (AUC = %0.2f)' % roc_auc_cec)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=font_size)
plt.ylabel('True Positive Rate', fontsize=font_size)
#plt.title('ROC curve for Palm')
plt.legend(loc="lower right", fontsize=font_size-1)
plt.tick_params(labelsize=font_size)
plt.colorbar(scalarmappaple)
plt.savefig('../output/figs/palm_ROC.png', bbox_inches='tight')  
plt.show()

In [ ]:
lr_precision_APL, lr_recall_APL, _ = precision_recall_curve(testy, probs_apl)
lr_precision_Basic, lr_recall_Basic, _ = precision_recall_curve(testy, probs_basic)
lw = 2
# plot the precision-recall curves
no_skill = len(testy[testy==1]) / len(testy)
plt.figure(figsize=(6, 5))
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', lw=lw, color='navy')
plt.plot([0, lr_recall_APL[lr_recall_APL > 0.01][-1]], [1, lr_precision_APL[lr_recall_APL>0.01][-1]], lw = lw, color='darkgreen')
plt.plot(lr_recall_APL[lr_recall_APL > 0.01], lr_precision_APL[lr_recall_APL > 0.01], lw = lw, label='APL', color='darkgreen')
plt.plot([0, lr_recall_Basic[lr_recall_Basic > 0.01][-1]], [1, lr_precision_Basic[lr_recall_Basic>0.01][-1]], lw = lw, color='darkorange')
plt.plot(lr_recall_Basic[lr_recall_Basic > 0.01], lr_precision_Basic[lr_recall_Basic > 0.01], lw = lw, label='Basic', color='darkorange')
plt.scatter(rec_list, prec_list, s = np.sqrt(nums_list)*3, label='Annotators', c= colormap(normalize(val_list)))
# axis labels
plt.xlabel('Recall', fontsize=font_size)
plt.ylabel('Precision', fontsize=font_size)
plt.colorbar(scalarmappaple)
plt.tick_params(labelsize=font_size)
# show the legend
plt.legend(loc='lower right', fontsize=font_size)
plt.savefig('../output/figs/palm_prec.png', bbox_inches='tight')  
# show the plot
plt.show()

In [ ]:
# cut-off selection 
n_thresh = 100
thresh_array = np.zeros(n_thresh)
IoU_array_apl = np.zeros(n_thresh)
acc_array_apl = np.zeros(n_thresh)
IoU_array_basic = np.zeros(n_thresh)
acc_array_basic = np.zeros(n_thresh)
for i in range(n_thresh):
    thresh_apl = np.quantile(probs_apl, i / n_thresh)
    ## for apl
    y_apl = probs_apl > thresh_apl
    n_inter_apl = np.sum(y_apl * testy)
    n_union_apl = np.sum((y_apl + testy) > 0)
    n_true_apl = np.sum(y_apl * testy) + np.sum((1-y_apl) * (1-testy))
    thresh_array[i] = i / n_thresh
    IoU_array_apl[i] = n_inter_apl / n_union_apl
    acc_array_apl[i] = n_true_apl / len(testy)
    ## for basic
    thresh_basic = np.quantile(probs_basic, i / n_thresh)
    y_basic = probs_basic > thresh_basic
    n_inter_basic = np.sum(y_basic * testy)
    n_union_basic = np.sum((y_basic + testy) > 0)
    n_true_basic = np.sum(y_basic * testy) + np.sum((1-y_basic) * (1-testy))
    IoU_array_basic[i] = n_inter_basic / n_union_basic
    acc_array_basic[i] = n_true_basic / len(testy)
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(thresh_array, IoU_array_apl, lw = lw, label='APL', color='darkgreen')
plt.plot(thresh_array, IoU_array_basic, lw = lw, label='Basic', color='darkorange')
plt.xlabel('Cut-off', fontsize=font_size)
plt.ylabel('IoU', fontsize=font_size)
plt.legend(fontsize=font_size)
#plt.title('Cut-off vs. IoU for Palm')
plt.subplot(122)
plt.plot(thresh_array, acc_array_apl, lw = lw, label='APL', color='darkgreen')
plt.plot(thresh_array, acc_array_basic, lw = lw, label='Basic', color='darkorange')
plt.xlabel('Cut-off', fontsize=font_size)
plt.ylabel('Accuracy', fontsize=font_size)
#plt.title('Threshold vs. Accuracy for Palm')
plt.legend(fontsize=font_size)
plt.show()


In [ ]:
# use selected cut-off, plot scatterplot for IoU vs. Accuracy
loc_apl = np.argmax(IoU_array_apl)
loc_basic = np.argmax(IoU_array_basic)
#print('Thresholds:', thresh_array[loc_apl], thresh_array[loc_basic])
#print('F1 score:', f1_score(testy, (probs_apl > thresh_array[loc_apl])), f1_score(testy, (probs_basic > thresh_array[loc_basic])))
#print('IoU:', IoU_array_apl[loc_apl], IoU_array_basic[loc_basic])
#print('Accuracy:', acc_array_apl[loc_apl], acc_array_basic[loc_basic])
scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array([])
plt.figure(figsize=(6, 5))
plt.scatter(acc_array_apl[loc_apl], IoU_array_apl[loc_apl], s=150, c='darkgreen', marker='s', label='APL')
plt.scatter(acc_array_basic[loc_basic], IoU_array_basic[loc_basic], s=150, c='darkorange', marker='s', label='Basic')
plt.scatter(acc_list, IoU_list, s=np.sqrt(nums_list)*3, label='Annotators', c= colormap(normalize(val_list)))
plt.xlabel('Accuracy', fontsize=font_size)
plt.ylabel('IoU', fontsize=font_size)
plt.colorbar(scalarmappaple)
plt.tick_params(labelsize=font_size)
plt.legend(fontsize=font_size)
plt.savefig('../output/figs/palm_IoU.png', bbox_inches='tight')  
plt.show()